# Milestone 1 - EDA and Preprocessing data 

In [30]:
# Import the necessary libraries

import numpy as np
import pandas as pd
import math
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
# to ignore warninigs
import warnings
warnings.simplefilter('ignore')

# Load Dataset

In [2]:
# Load the dataset from the parquet file

accidentsDataFrame = pd.read_csv('1992_Accidents_UK.csv' , index_col = 0 )

accidentsDataFrame.head()

,accident_year,accident_reference,location_easting_osgr,location_northing_osgr,longitude,latitude,police_force,accident_severity,number_of_vehicles,number_of_casualties,...,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag,lsoa_of_accident_location
accident_index,,,,,,,,,,,,,,,,,,,,,
1992010SU0965,1992,010SU0965,522790.0,200820.0,NaN,NaN,Metropolitan Police,Slight,5,1,...,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
1992010SU0966,1992,010SU0966,515830.0,201260.0,NaN,NaN,Metropolitan Police,Slight,2,1,...,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
1992010SU0967,1992,010SU0967,521460.0,201640.0,NaN,NaN,Metropolitan Police,Slight,2,1,...,No physical crossing facilities within 50 metres,Darkness - no lighting,NaN,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
1992010SU0968,1992,010SU0968,521040.0,202020.0,NaN,NaN,Metropolitan Police,Serious,2,1,...,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1
1992010SU0969,1992,010SU0969,522750.0,200680.0,NaN,NaN,Metropolitan Police,Slight,3,1,...,No physical crossing facilities within 50 metres,Darkness - no lighting,Raining + high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range,-1


In [3]:
accidentsDataFrame.columns

Index(['accident_year', 'accident_reference', 'location_easting_osgr',
       'location_northing_osgr', 'longitude', 'latitude', 'police_force',
       'accident_severity', 'number_of_vehicles', 'number_of_casualties',
       'date', 'day_of_week', 'time', 'local_authority_district',
       'local_authority_ons_district', 'local_authority_highway',
       'first_road_class', 'first_road_number', 'road_type', 'speed_limit',
       'junction_detail', 'junction_control', 'second_road_class',
       'second_road_number', 'pedestrian_crossing_human_control',
       'pedestrian_crossing_physical_facilities', 'light_conditions',
       'weather_conditions', 'road_surface_conditions',
       'special_conditions_at_site', 'carriageway_hazards',
       'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident',
       'trunk_road_flag', 'lsoa_of_accident_location'],
      dtype='object')

# Data Preprocessing

## Observe Numerical Features

In [6]:
# Retrieve the numerical features
numerical_dataset = accidentsDataFrame.select_dtypes(include=['int64','float'])

numerical_dataset.head()

,accident_year,location_easting_osgr,location_northing_osgr,longitude,latitude,number_of_vehicles,number_of_casualties,local_authority_ons_district,local_authority_highway,speed_limit,lsoa_of_accident_location
accident_index,,,,,,,,,,,
1992010SU0965,1992,522790.0,200820.0,NaN,NaN,5,1,-1,-1,70.0,-1
1992010SU0966,1992,515830.0,201260.0,NaN,NaN,2,1,-1,-1,60.0,-1
1992010SU0967,1992,521460.0,201640.0,NaN,NaN,2,1,-1,-1,70.0,-1
1992010SU0968,1992,521040.0,202020.0,NaN,NaN,2,1,-1,-1,70.0,-1
1992010SU0969,1992,522750.0,200680.0,NaN,NaN,3,1,-1,-1,70.0,-1


#### Observe missing values in numerical features

In [7]:
#print the number of null values in each columns
values = numerical_dataset.isnull().sum().values
percentage = (numerical_dataset.isnull().sum().values / numerical_dataset.shape[0]) * 100
unique_number = numerical_dataset.nunique()

dic = { 'unique': unique_number, 'count' : values, 'percentage': percentage}
null_df = pd.DataFrame(  data = dic , index = numerical_dataset.columns)
null_df

,unique,count,percentage
accident_year,1,0,0.000000
location_easting_osgr,35072,304,0.130414
location_northing_osgr,45286,304,0.130414
longitude,0,233104,100.000000
latitude,0,233104,100.000000
number_of_vehicles,25,0,0.000000
number_of_casualties,30,0,0.000000
local_authority_ons_district,1,0,0.000000
local_authority_highway,1,0,0.000000
speed_limit,25,0,0.000000


#### handling missing variables

We can observe that the whole values are missing of columns longitude and latidue -- > we will drop those colums\
we can observe that accident year, local_authority_ons_idstrict, local_authority_highway, and Iso_of_accident_location are
a one value columns -- > we will drop them\
we can also see that the percentage of the missing values in location_easting_osgr, location_northing_osgr is very low so
-- > we will replace the null values with the mean


In [8]:
#drop the columns 
numerical_dataset_na_dropped = numerical_dataset.drop( columns= [ 'accident_year','latitude','longitude', 'local_authority_ons_district', 'local_authority_highway' , 'lsoa_of_accident_location'], axis = 1)

#replace the nan with mean
numerical_dataset_na_dropped[ ['location_easting_osgr','location_northing_osgr'] ] = numerical_dataset_na_dropped[ ['location_easting_osgr','location_northing_osgr'] ].fillna( numerical_dataset_na_dropped[ ['location_easting_osgr','location_northing_osgr'] ].mean() )

In [9]:
#print the number of null values in each columns
values = numerical_dataset_na_dropped.isnull().sum().values
percentage = (numerical_dataset_na_dropped.isnull().sum().values / numerical_dataset_na_dropped.shape[0]) * 100
unique_number = numerical_dataset_na_dropped.nunique()

dic = { 'unique': unique_number, 'count' : values, 'percentage': percentage}
null_df = pd.DataFrame(  data = dic , index = numerical_dataset_na_dropped.columns)
null_df

,unique,count,percentage
location_easting_osgr,35073,0,0.0
location_northing_osgr,45287,0,0.0
number_of_vehicles,25,0,0.0
number_of_casualties,30,0,0.0
speed_limit,25,0,0.0


#### Observe and handling Outliers

In [10]:
# dictionary to store the value of the upper and lower limit of each feature for later use
table = {}

for featureName in numerical_dataset_na_dropped.columns:


    print(f'column: {featureName}')
    # Get the feature description which include min , max , 25% , 75% and so on..
    featureDescription = numerical_dataset_na_dropped[featureName].describe()
    
    # Calculate the 1st , the 3rd quartiles and the IQR value
    firstQuantile = featureDescription['25%']
    thirdQuantile = featureDescription['75%']
    
    IQR = thirdQuantile - firstQuantile
    
    # get the max value and min value 
    # any value that is greater than the max or less than the min is considerd as outlier
    
    maxRange = thirdQuantile + 1.5 * IQR
    minRange = firstQuantile - 1.5 * IQR
    
    table[featureName] = [maxRange, minRange]
    
    print(f"The upper limit is {maxRange}")
    print(f"The lower limit is {minRange}")
    
    # print the number of outliers in each feature
    print( 'number of outliers',numerical_dataset_na_dropped.query(f'{featureName} > {maxRange} or {featureName} < {minRange}').shape[0] )
    print('--------------------------')

column: location_easting_osgr
The upper limit is 748720.0
The lower limit is 147840.0
number of outliers 199
--------------------------
column: location_northing_osgr
The upper limit is 731126.25
The lower limit is -151283.75
number of outliers 3676
--------------------------
column: number_of_vehicles
The upper limit is 3.5
The lower limit is -0.5
number of outliers 4908
--------------------------
column: number_of_casualties
The upper limit is 1.0
The lower limit is 1.0
number of outliers 50613
--------------------------
column: speed_limit
The upper limit is 80.0
The lower limit is 0.0
number of outliers 0
--------------------------


In [11]:
# Create a copy of the database to store the changes on it
numerical_dataset_after_outliers = numerical_dataset_na_dropped.copy()
        

for feature in numerical_dataset_after_outliers.columns:

    # get the lower limit and upper limit for the current feature
    upper_limit, lower_limit  = table[feature][0], table[feature][1]

    # get the median of the current feature
    median = math.ceil(numerical_dataset_after_outliers[numerical_dataset_after_outliers[feature].between(lower_limit,\
      upper_limit , inclusive = True)][feature].median())

    # replace outliers by the median
    numerical_dataset_after_outliers[feature] = np.where(numerical_dataset_after_outliers[feature] < lower_limit , median  , numerical_dataset_after_outliers[feature])
    numerical_dataset_after_outliers[feature] = np.where(numerical_dataset_after_outliers[feature] > upper_limit , median  , numerical_dataset_after_outliers[feature])


## Observe Categorical Features

In [12]:
# Retrieve the numerical features
categorical_dataset = accidentsDataFrame.select_dtypes(include=['object'])

categorical_dataset.head()

,accident_reference,police_force,accident_severity,date,day_of_week,time,local_authority_district,first_road_class,first_road_number,road_type,...,pedestrian_crossing_human_control,pedestrian_crossing_physical_facilities,light_conditions,weather_conditions,road_surface_conditions,special_conditions_at_site,carriageway_hazards,urban_or_rural_area,did_police_officer_attend_scene_of_accident,trunk_road_flag
accident_index,,,,,,,,,,,,,,,,,,,,,
1992010SU0965,010SU0965,Metropolitan Police,Slight,11/12/1992,Friday,16:50,Hertsmere,A(M),1.0,Dual carriageway,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - lights lit,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range
1992010SU0966,010SU0966,Metropolitan Police,Slight,15/12/1992,Tuesday,07:55,Hertsmere,A,5183.0,Single carriageway,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range
1992010SU0967,010SU0967,Metropolitan Police,Slight,11/12/1992,Friday,17:40,Hertsmere,Motorway,25.0,Dual carriageway,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - no lighting,NaN,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range
1992010SU0968,010SU0968,Metropolitan Police,Serious,04/12/1992,Friday,12:05,Hertsmere,Motorway,25.0,Dual carriageway,...,None within 50 metres,No physical crossing facilities within 50 metres,Daylight,Fine no high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range
1992010SU0969,010SU0969,Metropolitan Police,Slight,27/11/1992,Friday,18:25,Hertsmere,A(M),1.0,Dual carriageway,...,None within 50 metres,No physical crossing facilities within 50 metres,Darkness - no lighting,Raining + high winds,Wet or damp,None,None,Data missing or out of range,Data missing or out of range,Data missing or out of range


In [13]:
categorical_dataset.nunique()

accident_reference                             233104
police_force                                       51
accident_severity                                   3
date                                              366
day_of_week                                         7
time                                             1434
local_authority_district                          460
first_road_class                                    6
first_road_number                                3445
road_type                                           5
junction_detail                                    10
junction_control                                    5
second_road_class                                   7
second_road_number                               2794
pedestrian_crossing_human_control                   4
pedestrian_crossing_physical_facilities             7
light_conditions                                    6
weather_conditions                                  9
road_surface_conditions     

#### Observations

1- accident reference is a unique ID for each accident so We will drop it \
2- local_authority_district, first_road_number, secodn_road_number are of high cardinality so we will drop them \
3- urban_or_rural_area, did_police_officer_attend_scene_of_accident, trunk_road_flag columns have 1 value so we will drop them                                    

In [14]:
categorical_dataset.drop( columns=['accident_reference','local_authority_district','first_road_number','second_road_number','urban_or_rural_area','did_police_officer_attend_scene_of_accident','trunk_road_flag'], inplace = True)

#### handling missing values

In [15]:
#print the number of null values in each columns
values = categorical_dataset.isnull().sum().values
percentage = (categorical_dataset.isnull().sum().values / categorical_dataset.shape[0]) * 100
unique_number = categorical_dataset.nunique()

dic = { 'unique': unique_number, 'count' : values, 'percentage': percentage}
null_df = pd.DataFrame(  data = dic , index = categorical_dataset.columns)
null_df

,unique,count,percentage
police_force,51,0,0.000000
accident_severity,3,0,0.000000
date,366,2,0.000858
day_of_week,7,0,0.000000
time,1434,2,0.000858
first_road_class,6,0,0.000000
road_type,5,1093,0.468889
junction_detail,10,0,0.000000
junction_control,5,0,0.000000
second_road_class,7,0,0.000000


In [16]:
#the na values are very low so we will replace na with most frequent value 
categorical_dataset_without_na = categorical_dataset.fillna( categorical_dataset.mode().loc[0] )

#### Discretisation

1- we convert the date column into the corresponding quarter number \
2- we convert the time column into evening and morning

In [17]:
categorical_dataset_without_na['quarter'] = pd.to_datetime(categorical_dataset_without_na.date , format = "%d/%m/%Y").dt.month
categorical_dataset_without_na['quarter'] = categorical_dataset_without_na['quarter'].apply( lambda x : 1 if x <= 3 else 2 if x <=6 else 3 if x <=9 else 4 )
categorical_dataset_without_na.drop(columns=['date'], inplace=True)

In [18]:
categorical_dataset_without_na['time'] = categorical_dataset_without_na.time.apply( lambda x: 'Morning' if  0 <= int( str(x).split(':')[0] ) < 12 else 'Evening'  )

### Encoding

In [19]:
# Apply label Encoding to accident_severity

# make a copy from the dataframe to modify on it
categorical_encoded = categorical_dataset_without_na.copy()

# make a dictionary to replace the serires values
accident_severity_dict = {'Slight' : 1 ,
                          "Serious" : 2,
                          "Fatal" : 3,
                          "." : 0
                        }


# replace the categorical values by the neumrical
categorical_encoded.accident_severity.replace(accident_severity_dict, regex=True , inplace = True)

In [20]:
# Apply label Encoding to  day_of_week


# make a dictionary to replace the serires values
day_of_week_dict = {     'Saturday' : 1 ,
                          "Sunday" : 2,
                          "Monday" : 3,
                          "Tuesday" : 4,
                          "Wednesday" : 5,
                          "Thursday" : 6,
                          "Friday" : 7,
                          "." : 0
                        }

# replace the categorical values by the neumrical
categorical_encoded.day_of_week.replace(day_of_week_dict, regex=True , inplace = True)

In [21]:
# Apply one-hot encoder to the remaining features

features = categorical_encoded.columns

features_binary_encoded = []

# make a new column for catergories that represent 5% or more from the dataset
for feature in features :
    
    if feature in ['day_of_week','accident_severity']:
        continue
    # get the number of unique categories in the current feature
    unique_values = categorical_encoded[feature].nunique()
    
    # if the number of uniqu categroies in the feature is <= 6 we will apply one hot encoding
    
    if (unique_values <= 6):
       
        encoded = pd.get_dummies(categorical_encoded[feature],prefix = f'{feature}-', drop_first=True)
        categorical_encoded = pd.concat( [categorical_encoded,encoded], axis = 1)
        categorical_encoded.drop(columns=[feature],inplace = True)
        
    else:
        
        
        # init the encoder
        encoder = ce.BinaryEncoder(cols = feature , return_df = True )

        # fit the encoder to the data
        categorical_encoded = encoder.fit_transform(categorical_encoded)
        
        


In [22]:
df = pd.concat([numerical_dataset_after_outliers, categorical_encoded],axis = 1)

In [23]:
df.drop(columns=['location_easting_osgr','location_northing_osgr'],inplace=True)

# Building Models

### For Diagnostics
We used different hyperparameters for each model and perform grid search to find the best model \
We used cross validation with k = 5 to tune the hyperparameter

In [24]:
# Step 1: Prepare the Data
X = df.drop("accident_severity", axis=1)
y = df["accident_severity"]

# Perform one-hot encoding or label encoding on categorical variables if necessary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Neural Network Model

In [33]:
# Step 1: Define the parameter grid
param_grid = {
    'hidden_layer_sizes': [ (5, 10), (10, 10) ],
    'activation': ['relu'],
    'learning_rate_init': [ 0.001, 0.01, 0.1]
}

# Step 2: Choose a search method
grid_search = GridSearchCV(MLPClassifier(random_state=42), param_grid, cv=5, scoring='accuracy')

# Step 3: Perform cross-validation and hyperparameter tuning
grid_search.fit(X_train, y_train)

# Step 4: Fit and evaluate models
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Step 6: Retrain the model
best_model.fit(X_train, y_train)


# Perform cross-validation with best parameters
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cross_val_scores)

# Step 7: Evaluate on the test set
test_accuracy = best_model.score(X_test, y_test)
print("Test Accuracy:", test_accuracy)


Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (5, 10), 'learning_rate_init': 0.01}
Best Score: 0.8055750193408384
Cross-Validation Scores: [0.80554827 0.80557211 0.80557211 0.80559131 0.80559131]
Test Accuracy: 0.804384196302175


## Logistic Regression

In [34]:
# Create Logistic Regression model
model = LogisticRegression(multi_class='multinomial')

# Define the parameter grid for hyperparameters
param_grid = {'C': [0.1, 1.0, 10.0], 'penalty': ['l1', 'l2']}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Retrieve the best model and best parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Train the model with the best parameters
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Perform cross-validation with best parameters
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cross_val_scores)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Cross-Validation Scores: [0.80554827 0.80557211 0.80557211 0.80559131 0.80559131]
Accuracy: 0.804384196302175


## KNN

In [35]:
best_score = 0
best_k = 0

for k in range(3, 26, 3):
    # Create a KNN model with the current value of K
    knn = KNeighborsClassifier(n_neighbors=k)

    # Fit the model to the training data
    knn.fit(X_train, y_train)

    # Predict the labels for the cross validation data
    y_pred = knn.predict(X_test)

    # Calculate the accuracy on the cross validation data
    acc = accuracy_score(y_test, y_pred)
    
    # Print the accuracy
    print('K = {}: Accuracy = {}'.format(k, acc))
    if acc > best_score:
        best_score = max(best_score, acc)
        best_k = k

print('----------------------------')
print(f'high accuracy {best_score} using k: {best_k}' )

K = 3: Accuracy = 0.7571532752777659
K = 6: Accuracy = 0.7922440049761915
K = 9: Accuracy = 0.7934451546480202
K = 12: Accuracy = 0.8012526275149071
K = 15: Accuracy = 0.8009952382995152
K = 18: Accuracy = 0.8024108789841705
K = 21: Accuracy = 0.8022821843764746
K = 24: Accuracy = 0.8033117412380422
----------------------------
high accuracy 0.8033117412380422 using k: 24


## Naive Bayes

In [36]:
# Create Multinomial Naive Bayes model
model = MultinomialNB()

# Define the parameter grid for alpha values
param_grid = {'alpha': [0.1, 1.0, 10.0, 100.0]}

# Perform grid search with cross-validation
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Retrieve the best model and best parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Train the model with the best parameters
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Perform cross-validation with best parameters
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5)
print("Cross-Validation Scores:", cross_val_scores)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Cross-Validation Scores: [0.78352678 0.78517124 0.78569556 0.78757329 0.78330712]
Accuracy: 0.7831066878297799
